In [27]:
import numpy as np
import sys
import pathlib
from os.path import join
path_to_file = str(pathlib.Path().resolve())
dir_path = join(path_to_file)

sys.path.append(join(dir_path, "HelperFiles"))
from helper import *
from rankshap import *
from train_models import *
from load_data import *

from os.path import join
data_path = join(dir_path, "Experiments", "Data")

import warnings
warnings.filterwarnings('ignore')

## Load dataset and train model

Git repo has built-in "bank", "brca", "census" (AKA adult), and "credit" datasets, and train_logreg() or train_neural_net() functions.

In [44]:
dataset = "brca" 
# X_train, y_train, X_test, y_test, mapping_dict = make_data(data_path, dataset)
# save_data(data_path, dataset, X_train, y_train, X_test, y_test, mapping_dict)
X_train, y_train, X_test, y_test, mapping_dict = load_data(data_path, dataset)

model = train_logreg(X_train, y_train)

print("Class imbalance: {}%".format(round(100*(max(np.mean(y_test), 1-np.mean(y_test))))))
Y_preds = (model(X_test) > 0.5).astype("int")
print("NN {}% accuracy".format(round(np.mean(Y_preds == y_test)*100)))
d = len(mapping_dict) if mapping_dict is not None else X_train.shape[1]
d

Class imbalance: 55%
NN 80% accuracy


20

## Preliminary run of Shapley sampling

Not infrequently, expected number of samples to be significant exceeds max number of perms; sampling it, though, it's significant. This suggests our distribution is too conservative. Maybe we can remove the factor of 2.

In [47]:
np.random.seed(1)
xloc = X_test[0:1]
# Defaults: n_samples_per_perm=2, n_init=100, max_n_perms=10000, n_equal=True, buffer=1.1, abs=True
shap_vals, diffs, converged = rankshap(model, X_train, xloc, K=2, alpha=0.2, mapping_dict=mapping_dict)

print("Converged: {}".format(converged))
ranking = get_ranking(shap_vals, abs=True)
print("Feature Ranking: {}".format(ranking))
n_perms_per_feature = [len(diffs[j]) for j in range(d)]
print("Number of perms per feature: {}".format(n_perms_per_feature))


Converged: True
Feature Ranking: [ 9  4  5  8 14 19  6 18 16 12 15  3  0 13  1 17  2 11 10  7]
Number of perms per feature: [100, 100, 100, 100, 4459, 100, 100, 100, 100, 4459, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]


# Simulate Experimental FWER

In [42]:
xloc = X_test[0:1]
N_runs = 50
K = 2
shap_vals_all = []
top_K = []
while len(top_K) < N_runs:
    shap_vals, diffs, converged = rankshap(model, X_train, xloc, K=2, alpha=0.2, mapping_dict=mapping_dict)
    if converged:
        est_top_K = get_ranking(shap_vals, abs=True)[:K]
        top_K.append(est_top_K)
        shap_vals_all.append(shap_vals)
        if len(top_K) % 5 == 0: print(len(top_K), calc_fwer(top_K))
    else:
        print("Failed to converge")

5 0.0
10 0.0
15 0.0
20 0.0
25 0.0
30 0.0
35 0.0
40 0.0
45 0.0
50 0.0


In [43]:
print(calc_fwer(top_K))

0.0
